In [1]:
import sys
sys.path.insert(0,'..')
import sib
import pandas as pd
import numpy as np

## Order of parameters

### Contacts:

in order: (i, j, t, lambda)

### Observations:
(i,s,t) which is in order:

- node
- state
- time

In [2]:
from pathlib import Path
import data_load

folder_data = Path("data_tree/")

params,contacts,observ,epidem = data_load.load_exported_data(folder_data)

n_inst = len(observ)
print("Number of instances: {}".format(len(observ)))
params

Number of instances: 50


{'type_graph': 'TREE',
 't_limit': 8,
 'lambda': 0.9,
 'mu': 0.3,
 'seed': 3,
 'n': 31,
 'p_tested': 0.01,
 'p_asymptomatic': 0.5,
 'p_delay_symptoms': {'1': 0.7, '2': 0.30000000000000004}}

In [3]:
contacts = contacts[["i","j","t","lambda"]]
#contacts.iloc[:30]

In [4]:
obs_all_df = []
for obs in observ:
    obs_df = data_load.convert_obs_to_df(obs)
    obs_df = obs_df[["i","st","t"]]
    obs_all_df.append(obs_df)

### Take one epidemic instance

In [8]:
inst = 8

In [9]:
epi_i = epidem[inst]
obs_dict = observ[inst]
print(obs_dict)

{'S': {'4': [27]}, 'I': {'2': [9], '3': [4], '4': [3, 19], '6': [2, 6, 7], '7': [5, 6, 16], '8': [12, 13]}, 'R': {}}


In [39]:
obs_df = obs_all_df[inst]

In [11]:
print(epi_i)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 1 1 0 0 0 0 1 2 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
 [2 1 1 1 1 0 0 1 1 1 2 0 0 0 0 0 0 0 0 1 1 2 2 0 0 0 0 0 0 0 0]
 [2 1 1 2 2 1 1 1 2 2 2 0 0 0 0 1 1 1 1 1 1 2 2 0 0 0 0 0 0 0 0]
 [2 1 1 2 2 1 1 1 2 2 2 1 1 1 1 1 1 2 1 1 1 2 2 0 0 0 0 0 0 0 0]
 [2 1 1 2 2 1 2 1 2 2 2 1 1 1 1 2 1 2 2 2 2 2 2 1 0 1 1 1 1 1 0]]


In [12]:
src = np.where(epi_i[0])[0][0]
print("Source is ",src)

Source is  20


### Setup sib

In [5]:
def callback(t, err, f):
    print(f"{t:4d}, {err:3.2e}", end="\r")

In [13]:
contacts_sib = list(contacts.to_records(index=False))
obs_sib = list(obs_df.to_records(index=False))

In [17]:
mu = params["mu"]
sib_pars = sib.Params(prob_r=sib.Gamma(mu=mu))
fg = sib.FactorGraph(sib_pars, contacts_sib, obs_sib)

In [18]:
sib.iterate(fg,100,3e-6,callback=callback)

In [29]:
beliefs = []
for n in fg.nodes:
    res = (np.array(n.bt),np.array(n.bg))
    beliefs.append(np.stack(res))
    #print(beliefs[-1].shape)

(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 11)
(2, 11)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)
(2, 10)


In [30]:
names = [str(i) for i in range(len(fg.nodes))]

In [32]:
np.savez(f"beliefs_inst_{inst}",**dict(zip(names,beliefs)))

In [33]:
data = np.load("beliefs_inst_8.npz")

In [34]:
data.files

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30']

## Batch processing

In [6]:
contacts_sib = list(contacts.to_records(index=False))
names = [str(i) for i in range(params["n"])]

In [7]:
all_marginals = []
for obs in obs_all_df:

    obs_sib = list(obs.to_records(index=False))
    mu = params["mu"]
    sib_pars = sib.Params(prob_r=sib.Gamma(mu=mu))
    fg = sib.FactorGraph(sib_pars, contacts_sib, obs_sib)

    sib.iterate(fg,100,3e-6,callback=callback)
    print("\nFinished")

    beliefs = []
    for n in fg.nodes:
        res = (np.array(n.bt),np.array(n.bg))
        beliefs.append(np.stack(res))
    all_marginals.append(tuple(beliefs))
    #print(beliefs[-1].shape)

   9, 0.00e+00
Finished
   8, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   8, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   8, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
Finished
   9, 0.00e+00
F

In [8]:
margin_names = [f"{inst}_{n}"  for inst in range(n_inst) for n in range(params["n"])]

In [9]:
dict_marg = {}
for inst, margist in enumerate(all_marginals):
    for n, marginals in enumerate(margist):
        dict_marg[f"{inst}_{n}"] = marginals

In [10]:
dict_marg["2_3"] == all_marginals[2][3]

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])

In [11]:
np.savez("beliefs_tree",**dict_marg)

In [26]:
num_nodes = params["n"]

In [32]:

def load_marginals(filename):
    data = np.load(filename)
    all_marg_load = []
    for inst in range(n_inst):
        margist = [data[f"{inst}_{n}"] for n in range(num_nodes)]
        all_marg_load.append(margist)
    return all_marg_load

In [33]:
all_marg_load = load_marginals("beliefs_tree.npz")

In [34]:
len(all_marg_load)

50

In [35]:
len(all_marg_load[0])

31